<a href="https://colab.research.google.com/github/DanLexOW/Healena/blob/main/Copy_of_%D0%94%D0%B0%D0%BD%D1%87%D0%B5%D0%BD%D0%BA%D0%BE_%D0%94%D0%975_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Устанавливаем библиотеки
install.packages("readxl")
install.packages("ggplot2")
install.packages("car")
install.packages("lmtest")

library(readxl)
library(ggplot2)
library(car)
library(lmtest)

#Загружаем таблицу
if (!dir.exists("plots")) dir.create("plots")
df <- read_excel("HR_Employees.xlsx", sheet = 1)

#1=
cat("\n 1. Описательная статистика \n")

str(df)
summary(df[, c("Age", "MonthlyIncome", "TotalWorkingYears", "YearsAtCompany")])
table(df$Gender)
table(df$Education)
table(df$Attrition)

cat("\n Выводы по данным \n")
cat("1. Количество наблюдений:", nrow(df), "\n")
cat("2. Средний доход:", round(mean(df$MonthlyIncome), 2),
    "(от", min(df$MonthlyIncome), "до", max(df$MonthlyIncome), ")\n")
cat("3. Возраст сотрудников: от", min(df$Age), "до", max(df$Age),
    "(средний", round(mean(df$Age), 2), ")\n")
cat("4. Общий стаж работы: от", min(df$TotalWorkingYears), "до", max(df$TotalWorkingYears),
    "(средний", round(mean(df$TotalWorkingYears), 2), ")\n")
cat("5. Доля уволившихся (Attrition = Yes):",
    round(mean(df$Attrition == "Yes") * 100, 2), "%\n")
cat("6. Пропусков в данных нет.\n")


#2
cat("\n 2. Предикторы \n")
cat("Для прогноза MonthlyIncome выбраны:\n")
cat("- Категориальные: Gender (пол), Education (уровень образования)\n")
cat("- Количественные: Age (возраст), TotalWorkingYears (общий стаж),\n")
cat("  YearsAtCompany (лет в текущей компании).\n")
cat("Почему они? - Эти переменные часто влияют на зарплату (человеческий капитал,\n")
cat("опыт, демография).\n")

df$Gender <- as.factor(df$Gender)
df$Education <- as.factor(df$Education)

#3
cat("\n 3. Корреляция \n")
cat("Для количественных используем корреляцию и графики рассеяния,\n")
cat("для категориальных – ящики с усами \n\n")

#Корреляция
cor_matrix <- cor(df[, c("MonthlyIncome", "Age", "TotalWorkingYears", "YearsAtCompany")],
                  use = "complete.obs")
cat("Корреляционная матрица:\n")
print(cor_matrix)

cat("\n Интерпретация корреляций \n")
cat("MonthlyIncome с Age: r =", round(cor_matrix["MonthlyIncome","Age"], 3),
    "- умеренная положительная связь.\n")
cat("MonthlyIncome с TotalWorkingYears: r =", round(cor_matrix["MonthlyIncome","TotalWorkingYears"], 3),
    "- сильная положительная связь.\n")
cat("MonthlyIncome с YearsAtCompany: r =", round(cor_matrix["MonthlyIncome","YearsAtCompany"], 3),
    "- умеренная положительная связь.\n")
cat("Все корреляции значимы (p < 0.05), что подтверждает наличие линейной зависимости.\n")

#Графики рассеяния
png("plots/scatter_quant.png", width = 800, height = 600)
par(mfrow = c(2, 2))
plot(df$Age, df$MonthlyIncome, main = "Age vs Income", xlab = "Age", ylab = "Income", col = "blue")
plot(df$TotalWorkingYears, df$MonthlyIncome, main = "TotalWorkYears vs Income", col = "blue")
plot(df$YearsAtCompany, df$MonthlyIncome, main = "YearsAtCompany vs Income", col = "blue")
dev.off()
cat("Графики рассеяния сохранены в plots/scatter_quant.png\n")

#Ящики с усами
png("plots/boxplot_gender.png")
boxplot(MonthlyIncome ~ Gender, df, main = "Income by Gender", col = c("pink", "lightblue"))
dev.off()
png("plots/boxplot_edu.png")
boxplot(MonthlyIncome ~ Education, df, main = "Income by Education", col = rainbow(5))
dev.off()
cat("Boxplot для Gender и Education сохранены.\n")
cat("Видно, что доход мужчин немного выше, и с ростом образования доход растёт.\n")

#4
cat("\n 4. Связь между предикторами \n")

#Корреляция количественных
cor_pred <- cor(df[, c("Age", "TotalWorkingYears", "YearsAtCompany")])
cat("Корреляция между Age, TotalWorkingYears, YearsAtCompany:\n")
print(cor_pred)

cat("\n Анализ мультиколлинеарности \n")
if (cor_pred["Age","TotalWorkingYears"] > 0.7) {
  cat("Age и TotalWorkingYears сильно коррелируют (r =", round(cor_pred["Age","TotalWorkingYears"], 3),
      "). Это может привести к мультиколлинеарности.\n")
} else {
  cat("Корреляции умеренные, мультиколлинеарность маловероятна.\n")
}

#Тест независимости категориальных
tbl <- table(df$Gender, df$Education)
chi_test <- chisq.test(tbl)
cat("\nХи-квадрат тест для Gender и Education:\n")
print(chi_test)
if (chi_test$p.value > 0.05) {
  cat("Gender и Education независимы (p =", round(chi_test$p.value, 3),
      "), мультиколлинеарности нет.\n")
} else {
  cat("Gender и Education связаны, но VIF покажет позже.\n")
}

#5
cat("\n Построение модели \n")
cat("Модель: MonthlyIncome ~ Age + TotalWorkingYears + YearsAtCompany + Gender + Education\n")
model <- lm(MonthlyIncome ~ Age + TotalWorkingYears + YearsAtCompany + Gender + Education, data = df)
summary(model)

#Сохраняем сводку для дальнейшего использования
s <- summary(model)

cat("\n Первичный анализ модели \n")
cat("Коэффициенты показывают влияние каждого фактора при фиксации остальных.\n")
cat("R² =", round(s$r.squared, 4), "– модель объясняет", round(s$r.squared*100, 2), "% дисперсии дохода.\n")
cat("F-статистика значима (p < 2.2e-16), модель в целом работает.\n")
cat("Age стал незначимым (p =", round(coef(s)["Age", "Pr(>|t|)"], 4),
    "), вероятно, из-за корреляции с TotalWorkingYears.\n")
cat("TotalWorkingYears – самый сильный предиктор (p < 2e-16).\n")
cat("Gender и Education незначимы, что может быть связано с тем, что их влияние\n")
cat("поглощается стажем.\n")

#6
cat("\n 6. Проверка допущений \n")
cat("Основные допущения линейной регрессии:\n")
cat("1. Отсутствие мультиколлинеарности (VIF)\n")
cat("2. Нормальность остатков (Q-Q plot, тест Шапиро-Уилка)\n")
cat("3. Гомоскедастичность (график остатков, тест Бройша-Пагана)\n")
cat("4. Отсутствие влиятельных наблюдений (расстояния Кука)\n\n")

#6.1 Мультиколлинеарность
cat("6.1 VIF (Variance Inflation Factor) \n")
vif_values <- vif(model)
print(vif_values)
if (all(vif_values[, "GVIF"] < 3)) {
  cat("Все VIF < 3 – мультиколлинеарность не критична.\n")
} else {
  cat("Есть VIF > 3 – возможна мультиколлинеарность, требуется анализ.\n")
}

#6.2 Нормальность остатков
cat("\n 6.2 Нормальность остатков \n")
res <- residuals(model)
png("plots/qqplot.png")
qqnorm(res, main = "Q-Q plot остатков")
qqline(res, col = "red")
dev.off()
shap <- shapiro.test(res)
cat("Тест Шапиро-Уилка: p =", shap$p.value, "\n")
if (shap$p.value > 0.05) {
  cat("Остатки распределены нормально (p > 0.05).\n")
} else {
  cat("Остатки не нормальны (p < 0.05). При большом объёме выборки (n=1470)\n")
  cat("это допустимо, но может влиять на точность доверительных интервалов.\n")
}

#6.3 Гомоскедастичность
cat("\n 6.3 Гомоскедастичность \n")
png("plots/resid_fitted.png")
plot(fitted(model), res, xlab = "Предсказанные значения", ylab = "Остатки",
     main = "График остатков")
abline(h = 0, col = "red")
dev.off()
bp <- bptest(model)
cat("Тест Бройша-Пагана: p =", bp$p.value, "\n")
if (bp$p.value > 0.05) {
  cat("Дисперсия остатков постоянна (гомоскедастичность).\n")
} else {
  cat("Обнаружена гетероскедастичность (p < 0.05). Это может приводить\n")
  cat("к неэффективности оценок.\n")
}

#6.4 Влиятельные наблюдения
cat("\n 6.4 Влиятельные наблюдения (Cook's distance) \n")
cooksd <- cooks.distance(model)
png("plots/cooks.png")
plot(cooksd, type = "h", main = "Расстояния Кука", ylab = "Cook's distance")
threshold <- 4 / (nrow(df) - length(coef(model)) - 1)
abline(h = threshold, col = "red")
dev.off()
infl <- sum(cooksd > threshold)
cat("Количество наблюдений с Cook's distance >", round(threshold, 4), ":", infl, "\n")
if (infl == 0) {
  cat("Влиятельных наблюдений нет.\n")
} else {
  cat("Обнаружено", infl, "влиятельных точек. Желательно проверить их на выбросы.\n")
}

#7
cat("\n 7. Значимость и R² \n")

f_stat <- s$fstatistic
f_pval <- pf(f_stat[1], f_stat[2], f_stat[3], lower.tail = FALSE)
cat("F-статистика =", f_stat[1], ", p-value =", f_pval, "\n")
if (f_pval < 0.05) {
  cat("Модель в целом значима (p < 0.05).\n")
} else {
  cat("Модель незначима.\n")
}
cat("Множественный R² =", round(s$r.squared, 4), "\n")
cat("Скорректированный R² =", round(s$adj.r.squared, 4), "\n")
cat("Это означает, что модель объясняет", round(s$r.squared*100, 2),
    "% дисперсии MonthlyIncome. Оставшиеся", round((1-s$r.squared)*100, 2),
    "% связаны с неучтёнными факторами.\n")

#8
cat("\n 8. Интерпретация \n")

coefs <- coef(model)
cat("Intercept (базовый уровень: женщина, Education = 1): ожидаемый доход =",
    round(coefs["(Intercept)"], 2), "\n")
cat("Age: при увеличении возраста на 1 год доход изменяется на",
    round(coefs["Age"], 2), "(p =", round(coef(s)["Age", "Pr(>|t|)"], 4),
    ") – незначимо.\n")
cat("TotalWorkingYears: дополнительный год общего стажа увеличивает доход на",
    round(coefs["TotalWorkingYears"], 2), "(p < 2e-16) – значимо.\n")
cat("YearsAtCompany: дополнительный год в текущей компании увеличивает доход на",
    round(coefs["YearsAtCompany"], 2), "(p =", round(coef(s)["YearsAtCompany", "Pr(>|t|)"], 4),
    ") – на грани значимости.\n")
cat("GenderMale: мужчины зарабатывают на", round(coefs["GenderMale"], 2),
    "больше женщин, но разница незначима (p =", round(coef(s)["GenderMale", "Pr(>|t|)"], 4), ")\n")

cat("\nОбразование (база – Below College):\n")
for (lev in levels(df$Education)[-1]) {
  est <- coefs[paste0("Education", lev)]
  pv <- coef(s)[paste0("Education", lev), "Pr(>|t|)"]
  cat("  Education", lev, ": разница =", round(est, 2), "(p =", round(pv, 4), ") –",
      ifelse(pv < 0.05, "значимо", "незначимо"), "\n")
}
cat("\nВывод: единственный устойчивый значимый фактор – общий трудовой стаж.\n")
cat("Возраст и образование теряют значимость при контроле стажа, что говорит\n")
cat("о том, что их влияние опосредовано накоплением опыта.\n")

#9
cat("\n 9. Стандартизованные коэффициенты \n")

#Ручная стандартизация количественных переменных
df_scaled <- df
df_scaled$Age <- scale(df$Age)
df_scaled$TotalWorkingYears <- scale(df$TotalWorkingYears)
df_scaled$YearsAtCompany <- scale(df$YearsAtCompany)
df_scaled$MonthlyIncome <- scale(df$MonthlyIncome)

model_scaled <- lm(MonthlyIncome ~ Age + TotalWorkingYears + YearsAtCompany + Gender + Education,
                   data = df_scaled)
beta <- coef(model_scaled)[-1]
print(beta)

max_beta <- names(which.max(abs(beta)))
cat("\nНаибольший по модулю стандартизованный коэффициент у переменной:", max_beta,
    "(", round(beta[max_beta], 4), ")\n")
cat("Это означает, что изменение этого предиктора на одно стандартное отклонение\n")
cat("вызывает наибольшее изменение дохода по сравнению с другими предикторами.\n")
cat("В нашем случае это TotalWorkingYears – общий стаж.\n")

#10
cat("\n 10. График предсказанных vs фактических \n")

pred <- fitted(model)
plot_data <- data.frame(actual = df$MonthlyIncome, pred = pred)

p <- ggplot(plot_data, aes(pred, actual)) +
  geom_point(alpha = 0.3, color = "blue") +
  geom_abline(slope = 1, intercept = 0, color = "red", linetype = 2, size = 1) +
  labs(title = "Предсказанные против фактических значений MonthlyIncome",
       x = "Предсказанные значения", y = "Фактические значения") +
  theme_minimal() +
  annotate("text", x = max(pred)*0.8, y = min(plot_data$actual)*0.9,
           label = paste("R² =", round(s$r.squared, 3)), size = 5, color = "darkred")
ggsave("plots/pred_vs_actual.png", plot = p, width = 6, height = 4)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘colorspace’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘urca’, ‘zoo’, ‘RcppArmadillo’, ‘cowplot’, ‘Deriv’, ‘forecast’, ‘microbenchmark’, ‘rbibutils’, ‘numDeriv’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘reformulas’, ‘Rdpack’, ‘RcppEigen’, ‘carData’, ‘abind’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: carData

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric





================= ЗАДАНИЕ 1 =================
Описательная статистика позволяет понять структуру данных, выявить
выбросы, пропуски и основные характеристики переменных.

tibble [1,470 × 34] (S3: tbl_df/tbl/data.frame)
 $ Age                     : num [1:1470] 34 49 27 39 36 26 31 39 50 29 ...
 $ Attrition               : chr [1:1470] "No" "No" "No" "No" ...
 $ BusinessTravel          : chr [1:1470] "Travel_Rarely" "Travel_Frequently" "Travel_Rarely" "Travel_Rarely" ...
 $ DailyRate               : num [1:1470] 628 1023 155 613 884 ...
 $ Department              : chr [1:1470] "Research & Development" "Sales" "Research & Development" "Research & Development" ...
 $ DistanceFromHome        : num [1:1470] 8 2 4 6 23 5 5 24 28 28 ...
 $ Education               : num [1:1470] 3 3 3 1 2 3 3 1 3 4 ...
 $ EducationField          : chr [1:1470] "Medical" "Medical" "Life Sciences" "Medical" ...
 $ EmployeeNumber          : num [1:1470] 2068 2065 2064 2062 2061 ...
 $ EnvironmentSatisfaction : n

      Age        MonthlyIncome   TotalWorkingYears YearsAtCompany  
 Min.   :18.00   Min.   : 1009   Min.   : 0.00     Min.   : 0.000  
 1st Qu.:30.00   1st Qu.: 2911   1st Qu.: 6.00     1st Qu.: 3.000  
 Median :36.00   Median : 4919   Median :10.00     Median : 5.000  
 Mean   :36.92   Mean   : 6503   Mean   :11.28     Mean   : 7.008  
 3rd Qu.:43.00   3rd Qu.: 8379   3rd Qu.:15.00     3rd Qu.: 9.000  
 Max.   :60.00   Max.   :19999   Max.   :40.00     Max.   :40.000  


Female   Male 
   588    882 


  1   2   3   4   5 
170 282 572 398  48 

...

Тест Бройша-Пагана: p = 2.720173e-60 
Обнаружена гетероскедастичность (p < 0.05). Это может приводить
к неэффективности оценок, рекомендуется использовать робастные ошибки.

--- 6.4 Влиятельные наблюдения (Cook's distance) ---


agg_record_1324372384 
                    2

Количество наблюдений с Cook's distance > 0.0027 : 123 
Обнаружено 123 влиятельных точек. Желательно проверить их на выбросы.

================= ЗАДАНИЕ 7 =================
Оцениваем, насколько модель статистически значима и какую долю вариации
дохода она объясняет.

F-статистика = 274.7873 , p-value = 7.3862e-285 
Модель в целом значима (p < 0.05).
Множественный R² = 0.6007 
Скорректированный R² = 0.5986 
Это означает, что модель объясняет 60.07 % дисперсии MonthlyIncome. Оставшиеся 39.93 % связаны с неучтёнными факторами.

================= ЗАДАНИЕ 8 =================
Интерпретация коэффициентов при фиксированных значениях остальных переменных.
Учитываем, что некоторые коэффициенты статистически незначимы.

Intercept (базовый уровень: женщина, Education = 1): ожидаемый доход = 1913.53 
Age: при увеличении возраста на 1 год доход изменяется на -19.42 (p = 0.1081 ) – незначимо.
TotalWorkingYears: дополнительный год общего стажа увеличивает доход на 469.69 (p < 2e-16) – значимо.
YearsAt

Warning message:
“Using `size` aesthetic for lines was deprecated in ggplot2 3.4.0.
ℹ Please use `linewidth` instead.”


График сохранён в plots/pred_vs_actual.png
Видно, что модель лучше предсказывает средние доходы, но хуже – крайние значения.

================= ОБЩИЙ ВЫВОД =================
Построенная модель объясняет 60% вариации дохода. Основной фактор – общий стаж работы.
Возраст, пол и образование не имеют самостоятельного значимого влияния при учёте стажа.
Обнаружена гетероскедастичность, что требует осторожности при интерпретации стандартных ошибок.
Для улучшения модели можно рассмотреть трансформацию переменных или включение взаимодействий.
Все графики сохранены в папку 'plots'.
